In [42]:
# import beautifulsoup4
import requests
from bs4 import BeautifulSoup
import pandas as pd

exercise_list={}

for k in list(range(1,	131)):
	page = requests.get("https://www.jefit.com/exercises/bodypart.php?id=11&exercises=All&search=&All=1&Bands=1&Bench=1&Dumbbell=1&EZBar=1&Kettlebell=1&MachineStrength=1&MachineCardio=1&Barbell=1&BodyOnly=1&ExerciseBall=1&FoamRoll=1&PullBar=1&WeightPlate=1&Other=1&Strength=0&Stretching=0&Powerlifting=0&OlympicWeightLifting=0&Beginner=0&Intermediate=0&Expert=0&page="+str(k))
	soup = BeautifulSoup(page.content, 'html.parser')
	titles = soup.find_all('h4')
	paragraphs = soup.find_all('p')
	pictures = soup.find_all('img')
	i = 0
	m = 2
	for j, title in enumerate(titles):
		page_dict = {}
		page_dict['Exercise'] = title.text
		page_dict['URL'] = title.find('a')['href']
		for l in range(i, i+3):
			page_dict.update({k: v[:-1] for k, v in (x.split(' : ') for x in [paragraphs[l].text])})
		i = i + 4
		page_dict['Picture_A'] = pictures[m]['src']
		page_dict['Picture_B'] = pictures[m+1]['src']
		m = m + 2
		exercise_list[str(k) + '-' + str(j)] = page_dict

exercises = pd.DataFrame.from_dict(exercise_list, orient='index').reset_index(drop=True)

In [43]:
for i, row in exercises.iterrows():
	row.Picture_A = row.Picture_A.replace('..', 'https://www.jefit.com')
	row.Picture_B = row.Picture_B.replace('..', 'https://www.jefit.com')
	row.URL = 'https://www.jefit.com/exercises/' + row.URL
exercises.to_csv('exercises.csv', index=True)
exercises.sample(10)

,Exercise,URL,Main Muscle Group,Type,Equipment,Picture_A,Picture_B
85,Barbell Jerk,https://www.jefit.com/exercises/414/Barbell-Jerk-,Shoulders,Olympic Weight Lifting,Barbell,https://www.jefit.com/images/exercises/1080_664/1656.jpg,https://www.jefit.com/images/exercises/1080_664/1657.jpg
214,Bodyweight Single-Leg Calf Raise,https://www.jefit.com/exercises/1240/Bodyweight-Single-Leg-Calf-Raise,Lower Legs,Strength,Body Only,https://www.jefit.com/images/exercises/1080_664/4944.jpg,https://www.jefit.com/images/exercises/1080_664/4945.jpg
115,Barbell Press on Stability Ball,https://www.jefit.com/exercises/1153/Barbell-Press-on-Stability-Ball,Shoulders,Strength,Barbell,https://www.jefit.com/images/exercises/1080_664/4612.jpg,https://www.jefit.com/images/exercises/1080_664/4613.jpg
1195,Stability Ball Weight Plate Pullover,https://www.jefit.com/exercises/188/Stability-Ball-Weight-Plate-Pullover,Triceps,Strength,Exercise Ball,https://www.jefit.com/images/exercises/1080_664/752.jpg,https://www.jefit.com/images/exercises/1080_664/753.jpg
917,Machine Reverse Fly,https://www.jefit.com/exercises/878/Machine-Reverse-Fly,Shoulders,Strength,Machine - Strength,https://www.jefit.com/images/exercises/1080_664/3512.jpg,https://www.jefit.com/images/exercises/1080_664/3513.jpg
105,Barbell Palms Down Wrist Curl,https://www.jefit.com/exercises/124/Barbell-Palms-Down-Wrist-Curl,Forearm,Strength,Barbell,https://www.jefit.com/images/exercises/1080_664/496.jpg,https://www.jefit.com/images/exercises/1080_664/497.jpg
787,Hanging Pike,https://www.jefit.com/exercises/59/Hanging-Pike,Abs,Strength,Pullup Bar,https://www.jefit.com/images/exercises/1080_664/236.jpg,https://www.jefit.com/images/exercises/1080_664/237.jpg
1168,Stability Ball Plank,https://www.jefit.com/exercises/979/Stability-Ball-Plank,Abs,Strength,Exercise Ball,https://www.jefit.com/images/exercises/1080_664/3916.jpg,https://www.jefit.com/images/exercises/1080_664/3917.jpg
314,Cable Rope Lat Pulldown,https://www.jefit.com/exercises/1244/Cable-Rope-Lat-Pulldown,Back,Strength,Machine - Strength,https://www.jefit.com/images/exercises/1080_664/4976.jpg,https://www.jefit.com/images/exercises/1080_664/4977.jpg
1291,Wide Grip Rear Pull-Up,https://www.jefit.com/exercises/85/Wide-Grip-Rear-Pull-Up,Back,Strength,Pullup Bar,https://www.jefit.com/images/exercises/1080_664/340.jpg,https://www.jefit.com/images/exercises/1080_664/341.jpg


In [177]:
def split_key(input):
	diccionario = {}
	for elemento in input:
		partes = elemento.split(':')
		if len(partes) == 2:
			clave, valor = partes
			diccionario[clave.strip()] = valor.strip()
		elif len(partes) == 3:
			clave1, intermedio, valor2 = partes
			valor1, clave2 = intermedio.split('  ')
			diccionario[clave1.strip()] = valor1.strip()
			diccionario[clave2.strip()] = valor2.strip()
	return diccionario

url_list = exercises.URL

data_list = {}

for i, url in enumerate(url_list):
	page = requests.get(url)
	soup = BeautifulSoup(page.content, 'html.parser')
	videos = soup.find_all('video')
	paragraphs = soup.find_all('div', class_="mt-2 p-2")
	img_list = soup.find_all('p', class_="text-center p-2")
	list_of_items = paragraphs[0].text.split('\n')
	data_list[i] = {}
	data_list[i]['url'] = url

	try:
		data_list[i]['Muscle'] = img_list[0].img.get('src')
	except:
		data_list[i]['Muscle'] = None
		
	try:
		data_list[i]['Video'] = videos[0].get('src')
	except:
		data_list[i]['Video'] = None
	
	data_list[i].update(split_key(list_of_items[1:2]))
	data_list[i].update(split_key(list_of_items[2:3]))
	data_list[i].update(split_key(list_of_items[4:5]))
	data_list[i].update(split_key(list_of_items[5:6]))
	data_list[i].update(split_key(list_of_items[6:7]))
	data_list[i].update(split_key(list_of_items[7:8]))
	data_list[i].update(split_key(list_of_items[8:9]))

data_list = pd.DataFrame.from_dict(data_list, orient='index').reset_index(drop=False)

In [178]:
for i, mus in enumerate(data_list['Muscle']):
	data_list.loc[i, 'Muscle'] = str(mus).replace('../../..', 'https://www.jefit.com')
data_list.to_csv('exercises_detail.csv', index=False)
data_list.sample(10)

,index,url,Muscle,Video,Main Muscle Group,Detailed Muscle Group,Type,Mechanics,Equipment,Difficulty
166,166,https://www.jefit.com/exercises/448/Barbell-Snatch-Shrug,https://www.jefit.com/images/exercises/b1.jpg,http://player.vimeo.com/external/160281276.sd.mp4?s=361be862287f40a054f81607aec3ea94624994ca&pro...,Back,Forearms,Olympic Weight Lifting,Compound,Barbell,Intermediate
690,690,https://www.jefit.com/exercises/884/Dumbbell-Shoulder-Press,https://www.jefit.com/images/exercises/b6.jpg,http://player.vimeo.com/external/160305306.sd.mp4?s=3c2b2b8e4cdd185dd31b39d7a4a90edb6da9463d&pro...,Shoulders,NaN,Strength,Compound,Dumbbell,Beginner
1194,1194,https://www.jefit.com/exercises/1205/Stability-Ball-Wall-Squat,https://www.jefit.com/images/exercises/b8.jpg,http://player.vimeo.com/external/160307234.hd.mp4?s=02538f1be97884ad40402f08762e91018381ad40&pro...,Upper Legs,NaN,Strength,Compound,Exercise Ball,Beginner
1224,1224,https://www.jefit.com/exercises/531/Sun-Salutation,https://www.jefit.com/images/exercises/b0.jpg,http://player.vimeo.com/external/160296936.sd.mp4?s=e5a7ea6ad22baa6611852d624bfb9e786c91714b&pro...,Abs,NaN,Stretching,Compound,Body Only,Beginner
906,906,https://www.jefit.com/exercises/836/Machine-Fly,https://www.jefit.com/images/exercises/b3.jpg,http://player.vimeo.com/external/160273548.sd.mp4?s=49dc7dcbf1502474af398310c49b9951f8b83f30&pro...,Chest,NaN,Strength,Isolation,Machine - Strength,Beginner
1096,1096,https://www.jefit.com/exercises/494/Smith-Machine-Stiff-Leg-Deadlift,https://www.jefit.com/images/exercises/b1.jpg,None,Back,Hamstrings,Strength,Compound,Machine - Strength,Beginner
966,966,https://www.jefit.com/exercises/335/Parallel-Bar-Hip-Raise,https://www.jefit.com/images/exercises/b0.jpg,http://player.vimeo.com/external/160280967.sd.mp4?s=68a0d670043e7662ffb211aa370f907e4c01c84e&pro...,Abs,Lower Abs,Strength,Isolation,Machine - Strength,Intermediate
416,416,https://www.jefit.com/exercises/973/Decline-Bench-Leg-Raise,https://www.jefit.com/images/exercises/b0.jpg,http://player.vimeo.com/external/160306852.sd.mp4?s=6453e2685886a558aab48855af920e24be96cd51&pro...,Abs,NaN,Strength,Isolation,Bench,Intermediate
481,481,https://www.jefit.com/exercises/1166/Dumbbell-Close-Grip-Press,https://www.jefit.com/images/exercises/b7.jpg,http://player.vimeo.com/external/160307177.hd.mp4?s=5df81e68ce0cb6993ee6d6fcf8de10557fbc4f0c&pro...,Triceps,NaN,Strength,Compound,Bench,Intermediate
907,907,https://www.jefit.com/exercises/45/Machine-Fly,https://www.jefit.com/images/exercises/b3.jpg,http://player.vimeo.com/external/160273548.sd.mp4?s=49dc7dcbf1502474af398310c49b9951f8b83f30&pro...,Chest,Outer Chest,Strength,Compound,Machine - Strength,Beginner


In [179]:
exercises_merge = pd.merge(exercises[['Exercise','URL','Picture_A','Picture_B']], data_list, right_on='url', left_on='URL', how='left')

exercises_merge.drop(columns=['url','index'], inplace=True)

exercises_merge.to_csv('exercises_merge.csv', index=True)

In [13]:
exercises_merge['Main Muscle Group'].drop_duplicates().to_csv('main_muscle_group.csv', index=False)
exercises_merge['Detailed Muscle Group'].drop_duplicates().to_csv('detailed_muscle_group.csv', index=False)